# Информация о географическом расположении клиентов (geolocation_olist_public_dataset.csv)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import seaborn as sns
import sys, os
import pathlib
from pathlib import Path
from pydantic import ValidationError
from olist_churn_prediction.paths import SRC_DIR

In [ ]:
sys.path.append(str(SRC_DIR))

In [ ]:
from olist_churn_prediction import feature_engineering
from olist_churn_prediction.schemas import GeolocationSchema
from olist_churn_prediction.paths import RAW_DIR

In [3]:
geolocation_data = feature_engineering.load_data(RAW_DIR / "geolocation_olist_public_dataset.csv",
                      schema = GeolocationSchema,
                      validate = True) 

In [6]:
display(geolocation_data.head())
geolocation_data.shape
geolocation_data.dtypes

,zip_code_prefix,city,state,lat,lng
0,10,sao paulo,sp,-23.547808,-46.636124
1,10,sao paulo,sp,-23.546082,-46.644821
2,10,sao paulo,sp,-23.540983,-46.642567
3,10,sao paulo,sp,-23.546759,-46.645771
4,10,sao paulo,sp,-23.546362,-46.643074


zip_code_prefix             Int64
city               string[python]
state              string[python]
lat                       float32
lng                       float32
dtype: object

## Трактовка признаков:

*zip_code_prefix* - это префикс почтового индекса. До 3 цифр;

*city* - город заказчика;

*state* - штат заказчика;

*lat* - анг. latitude, широта. Координаты ненастоящие, созданы случайно на основе принадлежности к определенному zip_code_prefix;

*lng* - анг. longitude, долгота. Также ненастоящие.

In [12]:
print('Пропуски:', geolocation_data.isna().sum())

Пропуски: zip_code_prefix    0
city               0
state              0
lat                0
lng                0
dtype: int64


In [13]:
for col in geolocation_data.columns:
    print(f"--- {col} ---")
    print(geolocation_data[col].value_counts())
    print()

--- zip_code_prefix ---
zip_code_prefix
10     500
378    500
394    500
392    500
391    500
      ... 
947      4
300      2
325      1
907      1
704      1
Name: count, Length: 851, dtype: int64

--- city ---
city
sao paulo            25001
curitiba             11605
rio de janeiro       10020
belo horizonte        8500
porto alegre          7357
                     ...  
varzea branca            1
sao braz do piaui        1
pedro laurentino         1
julio borges             1
muliterno                1
Name: count, Length: 5419, dtype: int64

--- state ---
state
sp    81929
mg    39851
rs    34831
pr    31441
rj    29057
ba    20463
go    11497
pe    11460
sc     9342
ce     7690
df     5645
pa     5237
ms     4921
es     4421
mt     4387
ma     3823
pb     2847
rn     2640
pi     2509
to     2394
al     1875
se     1448
ro     1001
am      807
ap      500
rr      500
ac      500
Name: count, dtype: int64

--- lat ---
lat
-12.147031    32
-22.384610    28
-23.466590    27
-23.5

In [8]:
print(len(geolocation_data['city'].unique()))
print(len(geolocation_data['state'].unique()))

5419
27


# Выводы из таблицы geolocation data:
1. Из-за отсутствия order_id или customer_unique_id будет затруднительно сопоставить местоположение клиента и его заказы. Возможно в будущем найдется способ это сделать.
2. В таблице 5419 уникальных городов и 27 уникальных штатов.
3. Возможно добаления новых признаков, основанных на расстоянии между продавцом и покупателем, хотя пока что нам неизвестны id покупателей.